# InGaN/GaN UV PL analysis

Input: filename of UV PL h5 file to analyze 
Output: 

In [7]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
import h5py as h5
import pandas as pd 
import IPython
import hyperspy.api as hs
import hyperspy_gui_ipywidgets
import hyperspy.signals as sgl
import ctc_funcs as fncn
import matplotlib
import scipy as sp
import seaborn as sns
from qtpy import QtWidgets
import os

def gui_fname(dir=None):
    """Select a file via a dialog and return the file name"""
    if dir is None: dir = '../'
    fname = QtWidgets.QFileDialog.getOpenFileName(None, "select data file...", dir, filter="h5 files (*asi_OO_hyperspec_scan.h5)")
    return fname[0]

## Loading data from file

In [8]:
fname = gui_fname()
fbase = os.path.splitext(fname)[0]
sns.set()
sns.set_style('ticks')

In [9]:
(sample, wls, spec_map, h_array, v_array, nh, nv, nf, dh, dv) = fncn.load_uvpl_map(fname)

Loading UVPL scan: D:/Chris/Google Drive/CoSES/Data/UV PL/25K 548 uGaN qw 5um 3 10kHz/190325_132351_asi_OO_hyperspec_scan.h5
Sample: 25K 548 uGaN qw 5um 3 10kHz 
Correcting negative minimum value in spec map: -3.0
43 x 42 spatial x 1044 spectral points


In [10]:
(En, En_spec_map, ne) = fncn.rebin_spec_map(spec_map, wls,spec_min=330,spec_max=800)

Interpolating spectral data from nm to eV
330.5604670320708 to 800.2132730178669 nm
43 x 42 spatial x 610 spectral points


## Plot data vs wavelength

In [13]:
f = fncn.plot_all_data(wls,spec_map,linlog=False)
plt.xlabel('Wavelength (nm)')
plt.ylabel('Spectrometer CCD counts (a.u.)')
plt.title(sample + ': summed spectrum');

In [14]:
# Use this function to plot summed maps in the spectral regions corresponding to PL, Si Raman, and WS2 Raman
f2 = fncn.plot_spec_bands(spec_map,wls,v_array,h_array,sample,' [nm] spec bands',
            (330,355,'plasma','laser'),
            (355,384,'plasma','GaN'),
            (384,410,'plasma','InGaN 1'),
            (410,480,'plasma','InGaN 2'),
            (480,650,'plasma','defect'),
            no_of_cols=3,
            percentile=3)

plt.tight_layout(h_pad=1)
f2.set_size_inches(7,7)

Desired spectral range: 330 to 355
Integrating indicies from 179 to 210
Desired spectral range: 355 to 384
Integrating indicies from 210 to 247
Desired spectral range: 384 to 410
Integrating indicies from 247 to 280
Desired spectral range: 410 to 480
Integrating indicies from 280 to 370
Desired spectral range: 480 to 650
Integrating indicies from 370 to 591


## Plot data vs eV

In [15]:
f3 = fncn.plot_all_data(En,En_spec_map,linlog=False)
plt.xlabel('$E$ (eV)')
plt.ylabel('I($E$) (a.u.)')
plt.title(sample + ': summed spectrum');

In [16]:
# Use this function to plot summed maps in the spectral regions corresponding to PL, Si Raman, and WS2 Raman
f4 = fncn.plot_spec_bands(En_spec_map,En,v_array,h_array,sample,' [eV] spec bands',
                          (1.5,3.6,'viridis','all'),
                          (1.5,2.5,'viridis','defect'),
                          (2.5,3.0,'viridis','InGaN 2'),
                          (3.0,3.2,'viridis','InGaN 1'),
                          (3.2,3.5,'viridis','GaN'),
                          (3.5,3.6,'viridis','laser'),
            no_of_cols=3,percentile=3)
plt.tight_layout(h_pad=1)
f4.set_size_inches(7,7)

Desired spectral range: 1.5 to 3.6
Integrating indicies from 0 to 568
Desired spectral range: 1.5 to 2.5
Integrating indicies from 0 to 263
Desired spectral range: 2.5 to 3.0
Integrating indicies from 263 to 402
Desired spectral range: 3.0 to 3.2
Integrating indicies from 402 to 457
Desired spectral range: 3.2 to 3.5
Integrating indicies from 457 to 540
Desired spectral range: 3.5 to 3.6
Integrating indicies from 540 to 568


In [17]:
f5 = fncn.plot_spec_bands(En_spec_map,En,v_array,h_array,sample,' [eV] 0.1 eV spec bands',
            (3.3,3.4,'viridis','1'),
            (3.2,3.3,'viridis','2'),
            (3.1,3.2,'viridis','3'),
            (3.0,3.1,'viridis','4'),
            (2.9,3.0,'viridis','5'),
            (2.8,2.9,'viridis','6'),
            (2.7,2.8,'viridis','7'),
            (2.6,2.7,'viridis','8'),
            (2.5,2.6,'viridis','9'),
            no_of_cols=5,percentile=3)

f5.set_size_inches(10,7)
plt.tight_layout(h_pad=1)

Desired spectral range: 3.3 to 3.4
Integrating indicies from 485 to 512
Desired spectral range: 3.2 to 3.3
Integrating indicies from 457 to 485
Desired spectral range: 3.1 to 3.2
Integrating indicies from 429 to 457
Desired spectral range: 3.0 to 3.1
Integrating indicies from 402 to 429
Desired spectral range: 2.9 to 3.0
Integrating indicies from 374 to 402
Desired spectral range: 2.8 to 2.9
Integrating indicies from 346 to 374
Desired spectral range: 2.7 to 2.8
Integrating indicies from 319 to 346
Desired spectral range: 2.6 to 2.7
Integrating indicies from 291 to 319
Desired spectral range: 2.5 to 2.6
Integrating indicies from 263 to 291


## Hyperspy signal setup

In [18]:
#data_red = fncn.bin_ndarray(En_spec_map[:,:,0:449], (45,52,450), operation='mean')
data_red = En_spec_map[:,:,:]
escale=En[1]-En[0]
eoffset=En[0]

# define signal and dimensions
s=hs.signals.Signal1D(data_red)
s.change_dtype('float64')
s.axes_manager[2].scale=escale
s.axes_manager[2].offset=eoffset
s.axes_manager[2].name='E'
s.axes_manager[2].units='eV'
s.axes_manager[0].name='x'
s.axes_manager[0].units='um'
s.axes_manager[0].scale=dh
s.axes_manager[1].name='Y'
s.axes_manager[1].units='um'
s.axes_manager[1].scale=dv

In [ ]:
# plot an interactive map with spectra at each location
roi = hs.roi.RectangularROI(left=0, right=100., top=0, bottom=100.)
s.plot()
scrop = roi.interactive(s)

In [ ]:
scrop.plot()

In [ ]:
s = scrop

## PCA with SVD, BSS

In [ ]:
# perform principal component analysis, look at the explained variance
s.decomposition(algorithm='svd')
s.plot_explained_variance_ratio()

In [ ]:
# take a look at orthogonalized 
s.blind_source_separation(number_of_components=5,on_loadings=True,max_iter=1000)
#s.plot_bss_results()
#bss_comp_arr = s.get_bss_loadings().split()

In [ ]:
#fncn.plot_bss_results(s,cmap='viridis')
fncn.plot_hs_results(s.get_bss_loadings(),s.get_bss_factors(),fncn.get_hs_axes(s),title='BSS',cmap='viridis',no_of_bands=5)

## PCA with NMF

In [ ]:
s.decomposition(algorithm='nmf',output_dimension=5)
#s.plot_decomposition_results()
nmf_comp_arr = s.get_decomposition_loadings().split()

In [ ]:
#fncn.plot_decomp_results(s,cmap='viridis')
fncn.plot_hs_results(s.get_decomposition_loadings(),s.get_decomposition_factors(),fncn.get_hs_axes(s),title='NMF',cmap='viridis',no_of_bands=5)

## Set up mask

In [ ]:
mask = None

In [ ]:
mask_data = nmf_comp_arr[1]
mask_data.plot()

In [ ]:
mask = mask_data > 1000
mask.plot()

## Correlations between BSS and NMF loadings

In [ ]:
fncn.plot_loading_correlations(s,mask=mask.data,alpha=0.1)
plt.gcf().set_size_inches(10,10)
plt.tight_layout(h_pad=1)

In [ ]:
fncn.plot_relevant_correlations(s,mask=mask.data,alpha=0.1)

In [ ]:
import matplotlib.backends.backend_pdf
pdf = matplotlib.backends.backend_pdf.PdfPages(fbase+".pdf")
for fig in range(1, plt.figure().number): ## will open an empty extra figure :(
    pdf.savefig( fig )
pdf.close()
plt.close(fig='all')

In [ ]:
print('test')

In [ ]:
sc = s.get_decomposition_model()
sc.plot()

In [ ]:
(s-sc).plot()

In [ ]:
f5 = fncn.plot_spec_bands(sc.data,En,v_array,h_array,sample,' [eV] 0.1 eV spec bands',
            (3.3,3.4,'plasma','1'),
            (3.2,3.3,'plasma','2'),
            (3.1,3.2,'plasma','3'),
            (3.0,3.1,'plasma','4'),
            (2.9,3.0,'plasma','5'),
            (2.8,2.9,'plasma','6'),
            (2.7,2.8,'plasma','7'),
            (2.6,2.7,'plasma','8'),
            (2.5,2.6,'plasma','9'),
            no_of_cols=5,percentile=3)
plt.tight_layout(h_pad=1)
f4.set_size_inches(7,7)

In [ ]:
plt.tight_layout(h_pad=1)

In [ ]:
plt.gcf()

In [ ]:
plt.figure()
aa = np.linspace(1e-3,20e-3,num=50)
dd = 135*np.tan(aa/10)
print(aa.shape)
plt.plot(aa,dd)

In [ ]:
plt.close(fig='all')

In [ ]:
scrop.plot()

In [ ]:
# perform principal component analysis, look at the explained variance
scrop.decomposition(algorithm='svd')
scrop.plot_explained_variance_ratio()

In [ ]:
# take a look at orthogonalized 
scrop.blind_source_separation(number_of_components=8,on_loadings=True,max_iter=1000)
fncn.plot_hs_results(scrop.get_bss_loadings(),scrop.get_bss_factors(),fncn.get_hs_axes(scrop),title='BSS',cmap='viridis')

In [ ]:
np.shape(fncn.get_hs_axes(s)[2])

In [ ]:
np.shape(fncn.get_hs_axes(scrop)[1])